# Day 22: Crab Combat

https://adventofcode.com/2020/day/22

In [53]:
def readDecks(filename):
    with open(filename) as f:
        decks = [ p.split(":\n")[1].split("\n") for p in f.read().split("\n\n")] 
    decks[1] = decks[1][:-1] # remove last \n in 2nd deck
    dint = []
    for d in decks:
        d = [ int(c) for c in d ]
        dint.append(d)
    return list(dint)

In [54]:
decks = readDecks("data/day22test0.txt")
decks

[[9, 2, 6, 3, 1], [5, 8, 4, 7, 10]]

## Part 1

In [62]:
def singleCombat(decks):
    iwin = -1
    iround = 0
    while True:
        iround +=1
        c1 = decks[0].pop(0)
        c2 = decks[1].pop(0)
        if c1 > c2:
            decks[0].append(c1)
            decks[0].append(c2)
        else:
            decks[1].append(c2)
            decks[1].append(c1)
        if len(decks[1])==0:
            print("Player 1 wins at round",iround)
            iwin = 0
            break
        if len(decks[0])==0:
            print("Player 2 wins at round",iround)
            iwin = 1
            break
    score = 0
    value = len(decks[iwin])
    for c in decks[iwin]:
        score += c*value
        value -= 1
    return score

decks = readDecks("data/input22.txt")
score = singleCombat(decks)
print("Score =",score)

Player 1 wins at round 1233
Score = 29764


## Part 2

* I need to remeber the status of previous rounds. Keeping a list of `int` and checking is probably not a good idea, I should use some kind of hashing.

* I need to count the lenght of each deck after a draw to chec whether the number of cards left corresponds to the number drawn.

In [81]:
decks = readDecks("data/day22test0.txt")

def deckToString(deck): # use string to hash deck
    return str(deck)[1:-1].replace(' ','')

def decksToString(decks):
    return keyDeck(decks[0])+"|"+keyDeck(decks[1])

decksToString(decks)

'9,2,6,3,1|5,8,4,7,10'

In [163]:
from collections import defaultdict 
from copy import deepcopy

def notPlayedBefore():
    return False

def recursiveCombat(_decks, _igame=1,debug=False):

    igame = _igame
    if debug:
        print("\n=== Game ",igame,"===")
    
    # make a (deep) copy of decks
    decks = deepcopy(_decks)
    
    # initialize round history for this game
    history = defaultdict(notPlayedBefore) 
    
    iround = 0
    
    while True:
        
        iwin = -1
        iround +=1
        
        if debug:
            print("\n-- Round",iround,"of game",igame,"--")
            print("Player 1's deck:",decks[0])
            print("Player 2's deck:",decks[1])
        
        # check game round history
        if history[decksToString(decks)]:
            if debug:
                print("Decks already seen before! Player 1 wins round",iround,"of game",igame)
            return 0,decks 
        else: # same deck status before round
            history[decksToString(decks)] = True
            
        # draw cards
        c1 = decks[0].pop(0)
        c2 = decks[1].pop(0)
        
        if debug:
            print("Player 1 plays:",c1)
            print("Player 2 plays:",c2)

        # If both players have at least as many cards remaining in their deck as the value 
        # of the card they just drew, the winner of the round is determined by playing a 
        # new game of Recursive Combat
        if len(decks[0])>=c1 and len(decks[1])>=c2:
            # To play a sub-game of Recursive Combat, each player creates a new deck by 
            # making a copy of the next cards in their deck (the quantity of cards copied 
            # is equal to the number on the card they drew to trigger the sub-game).
            if debug:
                print("Playing a sub-game to determine the winner...")
            newdecks = [decks[0][0:c1],decks[1][0:c2]]
            iwin, _ = recursiveCombat(newdecks,igame+1,debug)
            if debug:
                print("...anyway, back to game",igame)
        else:
            # Otherwise, at least one player must not have enough cards left in their deck to recurse
            # the winner of the round is the player with the higher-value card
            if c1 > c2:
                iwin = 0
            else:
                iwin = 1
        
        if iwin==-1: # round not assigned yet by previous conditions, use standard rules
            if c1 > c2:
                iwin = 0
            else:
                iwin = 1
            
        if iwin==0:
            if debug:
                print("Player 1 wins round",iround,"of game",igame,"!")
            decks[0].append(c1)
            decks[0].append(c2)
        else:
            if debug:
                print("Player 2 wins round",iround,"of game",igame,"!")
            decks[1].append(c2)
            decks[1].append(c1)
        
        if len(decks[1])==0:
            if debug:
                print("The winner of game",igame,"is player 1!")
            if igame==1:
                print("\n== Post-game results ==")
                print("Player 1's deck:", decks[0])
                print("Player 2's deck:", decks[1])
            return 0, decks

        if len(decks[0])==0:
            if debug:
                print("The winner of game",igame,"is player 2!")
            if igame==1:
                print("\n== Post-game results ==")
                print("Player 1's deck:", decks[0])
                print("Player 2's deck:", decks[1])
            return 1, decks

In [164]:
decks = readDecks("data/day22test0.txt")
iwin, final_decks = recursiveCombat(decks,debug=True)
score = 0
value = len(final_decks[iwin])
for c in final_decks[iwin]:
    score += c*value
    value -= 1
print("Score =",score)


=== Game  1 ===

-- Round 1 of game 1 --
Player 1's deck: [9, 2, 6, 3, 1]
Player 2's deck: [5, 8, 4, 7, 10]
Player 1 plays: 9
Player 2 plays: 5
Player 1 wins round 1 of game 1 !

-- Round 2 of game 1 --
Player 1's deck: [2, 6, 3, 1, 9, 5]
Player 2's deck: [8, 4, 7, 10]
Player 1 plays: 2
Player 2 plays: 8
Player 2 wins round 2 of game 1 !

-- Round 3 of game 1 --
Player 1's deck: [6, 3, 1, 9, 5]
Player 2's deck: [4, 7, 10, 8, 2]
Player 1 plays: 6
Player 2 plays: 4
Player 1 wins round 3 of game 1 !

-- Round 4 of game 1 --
Player 1's deck: [3, 1, 9, 5, 6, 4]
Player 2's deck: [7, 10, 8, 2]
Player 1 plays: 3
Player 2 plays: 7
Player 2 wins round 4 of game 1 !

-- Round 5 of game 1 --
Player 1's deck: [1, 9, 5, 6, 4]
Player 2's deck: [10, 8, 2, 7, 3]
Player 1 plays: 1
Player 2 plays: 10
Player 2 wins round 5 of game 1 !

-- Round 6 of game 1 --
Player 1's deck: [9, 5, 6, 4]
Player 2's deck: [8, 2, 7, 3, 10, 1]
Player 1 plays: 9
Player 2 plays: 8
Player 1 wins round 6 of game 1 !

-- Round 

In [166]:
decks = readDecks("data/input22.txt")
iwin, final_decks = recursiveCombat(decks,debug=False)
score = 0
value = len(final_decks[iwin])
for c in final_decks[iwin]:
    score += c*value
    value -= 1
print("Score =",score)


== Post-game results ==
Player 1's deck: [2, 6, 48, 36, 24, 20, 44, 23, 7, 5, 49, 13, 43, 9, 45, 29, 31, 8, 40, 21, 30, 19, 46, 28, 32, 14, 39, 38, 22, 3, 17, 4, 15, 11, 50, 42, 34, 18, 47, 35, 41, 16, 37, 10, 33, 26, 27, 25, 12, 1]
Player 2's deck: []
Score = 32588
